In [19]:
using DifferentialEquations
using StochasticDelayDiffEq
using SpecialFunctions
using Plots
using XLSX
using Distributions
using SimulatedAnnealingABC
using Distances
using DataFrames
using FFTW
using CSV

In [ ]:
# EXTRACTING OPEN MAGNETIC FLUX AND SUNSPOT NUMBER RECORDS FROM XLSX FILE

# Define DataFrame object
data = DataFrame(
  year = Int[],
  open_magn_flux = Float64[],
  open_magn_flux_err = Float64[],
  ssa_open_magn_flux = Float64[],
  sunspots_num = Float64[],
  sunspots_err = Float64[],
  ssa_sunspots = Float64[]
)

# Open file and for each row write data into the DataFrame
XLSX.openxlsx("SN Usoskin Brehm.xlsx") do file
  sheet = file["Data"] 

  for row in XLSX.eachrow(sheet)
    if isa(row[2], Number)
      push!(data, (
        year = row[2],
        open_magn_flux = row[3],
        open_magn_flux_err = row[4],
        ssa_open_magn_flux = row[5],
        sunspots_num = row[7],
        sunspots_err = row[8],
        ssa_sunspots = row[9]
      ))
    end
  end
end

In [2]:
# FUNCTIONS

# Box-shaped function for the magnetic field range 
function f(B, B_max = 10, B_min = 1)
  return 1 / 4 * (1 .+ erf.(B .^ 2 .- B_min ^ 2)) .* (1 .- erf.(B .^ 2 .- B_max ^ 2))
end

# Model function for the DDE
function MagneticField(du, u, h, p, t)
  N, T, tau, sigma, Bmax = p
  q = T / tau

  B, dB = u

  du[1] = dB
  du[2] = - ((2 / tau) * dB + (B / tau^2) + N * h(p, t - q)[1] * f(h(p, t - q)[1]))
end

# Noise function for the DDE
function noise!(du, u, h, p, t)
  N, T, tau, sigma, Bmax = p
  du[1] = (sigma * Bmax)
end

# Distance function in the sABC algorithm
function f_dist(θ::Vector{Float64}; type::Int64 = 1, indeces::Vector{Int64} = 1:6:120, fourier_data::Vector{Int64})
  prob = SDDEProblem(MagneticField, noise, B0, h, tspan, θ)
  sol = solve(prob, EM(), dt = 0.01)
  
  simulated_data = sol[1,:]
  fourier_transform = fft(simulated_data)
  fourier_stats = abs.(fourier_transform[indeces])

  rho = [euclidean(fourier_stats[i], fourier_data[i]) for i in 1:length(fourier_stats)]
  return rho
end

# function to create a new directory for each simulations, in order to store the needed files
function create_directory()
  base_path = pwd()
  base_path = joinpath(base_path, "Simulations")
  i = 1
  dir_name = "Simulation $i"
  dir_path = joinpath(base_path, dir_name)
  
  while isdir(dir_path)
    i += 1
    dir_name = "Simulation $i"
    dir_path = joinpath(base_path, dir_name)
  end
  
  mkpath(dir_path)
  println("Directory created at: $dir_path")
  cd(dir_path)
end

create_directory (generic function with 1 method)

In [3]:
# function to create a new directory for each simulations, in order to store the needed files
function create_directory()
  base_path = pwd()
  base_path = joinpath(base_path, "Simulations")
  i = 1
  dir_name = "Simulation $i"
  dir_path = joinpath(base_path, dir_name)
  
  while isdir(dir_path)
    i += 1
    dir_name = "Simulation $i"
    dir_path = joinpath(base_path, dir_name)
  end
  
  mkpath(dir_path)
  println("Directory created at: $dir_path")
  cd(dir_path)
end

create_directory (generic function with 1 method)

In [6]:
create_directory()

Directory created at: /mnt/c/Users/Utente/LCP_B/Project/Simulations/Simulation 1


In [4]:
function write_params_sim(par::Vector{Float64}, tspan::Tuple{Int, Int}, B0::Vector{Float64}, h::Function, noise0::Vector{Float64})
  open("Parameters.txt", "w") do io
      println(io, "Simulated data parameters:")
      println(io, "N = $(par[1])")
      println(io, "T = $(par[2])")
      println(io, "tau = $(par[3])")
      println(io, "sigma = $(par[4])")
      println(io, "B_max = $(par[5])\n")

      println(io, "Time:")
      println(io, "tspan = $tspan\n")
      
      println(io, "Initial conditions:")
      println(io, "B0 = $(B0)")
      println(io, "h(p, t) = $(string(h))")
      println(io, "noise0 = $(noise0)")
  end
end

write_params_sim (generic function with 1 method)

In [7]:
N = 6.2
T0 = 3
T1 = 0.1
tau = 3.5
T = T0 + T1
sigma = 0.04
B_max = 6.0

par = [N, T, tau, sigma, B_max]

# Define initial conditions
B0 = [3.0, 0.0]
tspan = (0, 1000)
h(p, t) = [0.0, 0.0]

# Define noise initial conditions
noise0 = [1.0]

write_params_sim(par, tspan, B0, h, noise0)

In [8]:
prob = SDDEProblem(MagneticField, noise!, B0, h, tspan, par)
sol = solve(prob, EM(), dt = 0.01)

retcode: Success
Interpolation: 1st order linear
t: 100002-element Vector{Float64}:
    0.0
    0.01
    0.02
    0.03
    0.04
    0.05
    0.060000000000000005
    0.07
    0.08
    0.09
    0.09999999999999999
    0.10999999999999999
    0.11999999999999998
    ⋮
  999.8999999992357
  999.9099999992357
  999.9199999992356
  999.9299999992356
  999.9399999992356
  999.9499999992356
  999.9599999992356
  999.9699999992356
  999.9799999992356
  999.9899999992356
  999.9999999992356
 1000.0
u: 100002-element Vector{Vector{Float64}}:
 [3.0, 0.0]
 [3.0167759151444047, -0.0024489795918367346]
 [3.018203861620667, -0.004897659639184936]
 [3.0354110837117005, -0.007333512899712587]
 [3.0204240297484546, -0.009769493710662557]
 [3.017795867978612, -0.012179320301498327]
 [3.006654937044492, -0.014573231016492714]
 [2.9993690207251547, -0.01694436760418989]
 [3.0049512680318538, -0.019296007153166074]
 [3.027183608830279, -0.02163876569843929]
 [3.0622488526249856, -0.0239862859016566]
 [3.089

In [36]:
function save_solution(sol::Union{RODESolution, Vector{RODESolution}})
  filename = "Simulated_sol.csv"
  CSV.write(filename, Dict("Time" => sol.t, "Solution" => Matrix(sol[1:end, :])))
  println("Solution saved to file: $filename")
end

function load_solution()
  filename = "Simulated_sol.csv"
  df = CSV.read(filename, DataFrame)


    # Extract the row corresponding to "Time"
    t_row = df[1, :]

    # Extract the row corresponding to "Solution"
    sol_row = df[2, :]

    # Convert the rows to arrays
    t = convert(Vector, t_row)
    sol = convert(Vector, sol_row)

    return t, sol
end

load_solution (generic function with 2 methods)

In [38]:
function save_solution(sol::Union{RODESolution, Vector{RODESolution}})
    filename = "Simulated_sol.csv"
    # Transpose the solution array
    solution_transposed = permutedims(Matrix(sol[1:end, :]), [2, 1])
    # Write the transposed data to the CSV file
    CSV.write(filename, DataFrame(Time = sol.t, Solution = solution_transposed))
    println("Solution saved to file: $filename")
end

save_solution (generic function with 4 methods)

In [39]:
save_solution(sol)

LoadError: TaskFailedException

[91m    nested task error: [39mArgumentError: adding AbstractArray other than AbstractVector as a column of a data frame is not allowed
    Stacktrace:
     [1] [0m[1m_preprocess_column[22m[0m[1m([22m[90mcol[39m::[0mMatrix[90m{Float64}[39m, [90mlen[39m::[0mInt64, [90mcopycols[39m::[0mBool[0m[1m)[22m
    [90m   @[39m [33mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/dataframe/[39m[90m[4mdataframe.jl:248[24m[39m
     [2] [0m[1m(::DataFrames.var"#198#200"{Bool, Vector{Any}, Int64})[22m[0m[1m([22m[0m[1m)[22m
    [90m   @[39m [33mDataFrames[39m [90m~/.julia/packages/DataFrames/58MUJ/src/dataframe/[39m[90m[4mdataframe.jl:223[24m[39m

In [37]:
t, sol = load_solution()

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mDataFrameRow{DataFrame, DataFrames.Index}[39m[0m to an object of type 
[0m  [91mVector[39m

[0mClosest candidates are:
[0m  convert(::Type{Vector}, [91m::Sundials.NVector[39m)
[0m[90m   @[39m [33mSundials[39m [90m~/.julia/packages/Sundials/jrHAF/src/[39m[90m[4mnvector_wrapper.jl:64[24m[39m
[0m  convert(::Type{Vector}, [91m::StatsBase.UnitWeights{T}[39m) where T
[0m[90m   @[39m [32mStatsBase[39m [90m~/.julia/packages/StatsBase/ebrT3/src/[39m[90m[4mweights.jl:317[24m[39m
[0m  convert(::Type{Vector}, [91m::StatsBase.AbstractWeights[39m)
[0m[90m   @[39m [32mStatsBase[39m [90m~/.julia/packages/StatsBase/ebrT3/src/[39m[90m[4mweights.jl:35[24m[39m
[0m  ...


In [27]:
function load_solution(filename::String = "Simulated_sol.csv")
  # Read the CSV file into a DataFrame
  df = CSV.read(filename, DataFrame)

  # Print column names and first few rows of the DataFrame
  println("Column names: ", names(df))
  println("First few rows:")
  println(first(df, 5))

  # Extract the time vector and solution array from the DataFrame
  t = df[!, 1]  # Assuming the time column is the first column
  sol = convert(Matrix, df[:, 2:end])

  return t, sol
end

load_solution (generic function with 2 methods)

In [28]:
t, sol = load_solution()

Column names: ["first", "second"]
First few rows:
2×2 DataFrame
 Row │ first     second                            
     │ String15  String                            
─────┼─────────────────────────────────────────────
   1 │ Solution  [3.0 3.0167759151444047 3.018203…
   2 │ Time      [0.0, 0.01, 0.02, 0.03, 0.04, 0.…


LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mDataFrame[39m[0m to an object of type 
[0m  [91mMatrix[39m

[0mClosest candidates are:
[0m  convert(::Type{Matrix}, [91m::BandedMatrices.BandedMatrix[39m)
[0m[90m   @[39m [35mBandedMatrices[39m [90m~/.julia/packages/BandedMatrices/dec3g/src/banded/[39m[90m[4mBandedMatrix.jl:754[24m[39m
[0m  convert(::Type{Matrix}, [91m::PooledArrays.PooledMatrix{T, R}[39m) where {T, R}
[0m[90m   @[39m [36mPooledArrays[39m [90m~/.julia/packages/PooledArrays/Vy2X0/src/[39m[90m[4mPooledArrays.jl:511[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  ...


In [ ]:
simulated_data = sol[1, :]
fourier_transform_sim = fft(simulated_data)
fourier_indeces = 1:6:120
fourier_sim = abs.(fourier_transform_sim[fourier_indeces])

Fs = 1 / ((sol.t[2] - sol.t[1]))
N = length(simulated_data)
freqs = fftfreq(N, Fs)

# Plot the magnitude spectrum
Plots.plot(freqs, abs.(fourier_transform_sim), xlabel="Frequency (Hz)", ylabel="Magnitude", title="Magnitude Spectrum", legend=false, xlims =(0, 1))

In [ ]:
# FUNCTION FOR THE DISTANCE IN THE SABC ALGORITHM

function f_dist(θ::Vector{Float64}; type::Int64 = 1)
  prob = SDDEProblem(MagneticField, noise, B0, h, tspan, θ)
  sol = solve(prob, EM(), dt = 0.01)
  
  simulated_data = sol[1,:]

  fourier_transform = fft(simulated_data)
  fourier_indeces = 1:6:120
  fourier_stats = abs.(fourier_transform[fourier_indeces])

  rho = [euclidean(fourier_stats[i], fourier_sim[i]) for i in 1:length(fourier_stats)]
  
  return rho
end

In [ ]:
# FUNCTION FOR THE PLOTTING OF THE POSTERIORS AS A CORNER PLOT

function post_plotting(post_par, true_vals)
  pairplot(
    posterior_params,
    PairPlots.Truth(
        (;
            N_value = true_vals[1],
            T_value = true_vals[2],
            tau_value = true_vals[3],
            sigma_value = true_vals[4],
            Bmax_value = true_vals[5]
        ),
        
        label="True Values"
    )
  )
end

In [ ]:
prior = product_distribution(Uniform(1, 15), Uniform(0.1, 15.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))

result = sabc(f_dist, prior, n_particles=1000, n_simulation=1000000, v=1.0, type = 1)

display(result)

In [ ]:
## Extract posterior population, trajectories for epsilon, rho and u
pop_singeps = hcat(result.population...)
eps_hist = vec(hcat(result.state.ϵ_history...))
u_hist = vec(hcat(result.state.u_history...))

p1 = Plots.plot(eps_hist, title="Epsilon History", xlabel="Iteration", legend=false, yscale=:log10)
p2 = Plots.plot(u_hist, title="U History", xlabel="Iteration", legend=false)

# Combine plots
combined_plot = Plots.plot(p1, p2, layout=(1, 2), size=(1000, 400))

display(combined_plot)

In [ ]:
rho_singeps = hcat(result.state.ρ_history...)

rho_plots = []

for i in 1:20
    push!(rho_plots, Plots.plot(rho_singeps[i, :], title="Rho History Stat $i", xlabel="Iteration", legend=false, yscale=:log10))
end

combined_plot = Plots.plot(rho_plots..., layout=(5, 4), size=(1200, 1800))

display(combined_plot)

In [ ]:
all_rho_plot = Plots.plot(1:size(rho_singeps, 2), rho_singeps', title="All Rho History", xlabel="Iteration", ylabel="Rho", legend=false, yscale=:log10)

In [ ]:
# Extract parameter samples for each parameter
pop_history = vec(pop_singeps)
param_samples = reshape(pop_history, 5, 1000)

N_value = param_samples[1, :]
T_value = param_samples[2, :]
tau_value = param_samples[3,:]
sigma_value = param_samples[4,:]
Bmax_value = param_samples[5, :]

posterior_params = DataFrame(;N_value, T_value, tau_value, sigma_value, Bmax_value)

In [ ]:
post_plotting(posterior_params, [6.2, 3.1, 3.5, 0.04, 6.0])